# COMS 4770 - Assignment 7 - Hedgren

In [1]:
import numpy as np
import cmath

### 1. (a) Implement a procedure **evalPoly(n, a[], t, v[])** which takes as input the n + 1 coefficients of a polynomial of degree n stored in the array a[]. Here a[n] is the leading coefficient. At a point t, the procedure produces the values of the polynomial and its derivative in v[0] and v[1], respectively. Your procedure should support complex numbers.

In [2]:
"""
Evaluates a polynomial and its derivative at point t.

Parameters:
n (int): Degree of the polynomial.
a (list): Coefficients of the polynomial, where a[n] is the leading coefficient.
t (complex or float): The point where the polynomial and derivative are evaluated.
v (list): Output list where v[0] will store p(t) and v[1] will store p'(t).
"""
def evalPoly(n, a, t, v):
    poly_val = complex(0, 0)
    deriv_val = complex(0, 0)

    for i in range(n + 1):
        poly_val += a[i] * (t ** (n - i))
        if i < n:
            deriv_val += a[i] * (n - i) * (t ** (n - i - 1))

    v[0] = poly_val
    v[1] = deriv_val

#### (b) (6 pts) Use the procedure to evaluate the polynomial $p(x) = x^8 − 170x^6 + 7392x^4 − 39712x^2 + 51200$ at $1.414214$ and $1 + 2i$. Also, obtain its derivatives at these two points.

In [3]:
n = 8
a = [1, 0, -170, 0, 7392, 0, -39712, 0, 51200]

points = [1.414214, 1 + 2j]
results = []

for t in points:
    v = [0, 0]
    evalPoly(n, a, t, v)
    results.append((t, v[0], v[1]))

for t, poly_value, deriv_value in results:
    print(f"At t = {t}:")
    print(f"p({t}) = {poly_value}")
    print(f"p'({t}) = {deriv_value}")
    print()

At t = 1.414214:
p(1.414214) = (-0.015041687198390719+0j)
p'(1.414214) = (-34371.01227099437+0j)

At t = (1+2j):
p((1+2j)) = (98175-343400j)
p'((1+2j)) = (-446260-177000j)



##### Outputs:
At $t = 1.414214$:
$$p(1.414214) = (-0.015041687198390719+0j)$$
$$p'(1.414214) = (-34371.01227099437+0j)$$

At $t = (1+2j)$:
$$p((1+2j)) = (98175-343400j)$$
$$p'((1+2j)) = (-446260-177000j)$$

*NOTE: python's complex() uses j for denoting imaginary numbers instead of i*

### 2. In this problem you are asked to implement the fast Fourier transform and polynomial multiplication. Your code should have at least the following components:

#### (a) (11 pts) A procedure **DFT(n, a[], ahat[])** which takes as input the coefficients a[0], a[1], . . ., a[n] of a polynomial of degree n, where a[n] is the leading coefficient. It generates an array ahat[] that stores the images of these coefficients under the discrete Fourier transform.

In [4]:
def DFT(n, a, ahat):
    omega_N = np.exp(2j * np.pi / n)
    
    for k in range(n):
        ahat_k = 0
        
        for j in range(n):
            ahat_k += a[j] * omega_N**(j * k)
        
        ahat[k] = ahat_k

#### (b) (6 pts) A procedure **IDFT(n, ahat[], a[])** which performs the inverse DFT. Namely, it outputs the coefficients a[0], a[1], . . ., a[n] from their images ahat[0], ahat[1], . . ., ahat[n] under DFT.

In [5]:
def IDFT(n, ahat, a):
    omega_N_inv = np.exp(-2j * np.pi / n)
    
    for j in range(n):
        a_j = 0
        
        for k in range(n):
            a_j += ahat[k] * omega_N_inv**(j * k)
        
        a[j] = a_j / n

In [6]:
# TESTING DFT and IDFT
n = 4
a = [1, 2, 3, 4]
ahat = [0] * (n)

a_new = [0] * 4

def round_coeff(coeffs, threshold=1e-15):
    rounded_coeffs = []
        
    for coeff in coeffs:
        real_part = np.round(coeff.real, 10)
        imag_part = np.round(coeff.imag, 10)
        
        if abs(imag_part) < threshold:
            imag_part = 0
        
        if imag_part == 0:
            rounded_coeffs.append(real_part)
        else:
            rounded_coeffs.append(complex(real_part, imag_part))
    
    return rounded_coeffs

DFT(n, a, ahat)
print("DFT coefficients rounded (ahat):", round_coeff(ahat))
IDFT(n, ahat, a_new)
print("IDFT coefficients rounded (a):", round_coeff(a_new))

print()
print("Difference between original a and a after IDFT rounded:")
result = [x - y for x, y in zip(a_new, a)]
print(round_coeff(result))

DFT coefficients rounded (ahat): [10.0, (-2-2j), -2.0, (-2+2j)]
IDFT coefficients rounded (a): [1.0, 2.0, 3.0, 4.0]

Difference between original a and a after IDFT rounded:
[0.0, 0.0, -0.0, 0.0]


#### (c) (5 pts) A procedure **multiplyPolys(n, m, a[], b[], c[])** which uses FFT to compute the product of two polynomials of degrees n and m, respectively. The coefficients of these two polynomials are stored in the arrays a[] and b[], respectively, with leading coefficients a[n] and b[m]. The coefficients of the product polynomial will be stored in the array c[].

In [7]:
def multiplyPolys(n, m, a, b):
    size = 1
    while size < n + m + 1:
        size *= 2
    
    a_padded = np.pad(a, (0, size - (n + 1)), 'constant')
    b_padded = np.pad(b, (0, size - (m + 1)), 'constant')
    
    A_hat = np.zeros(size, dtype=complex)
    B_hat = np.zeros(size, dtype=complex)
    
    DFT(size, a_padded, A_hat)
    DFT(size, b_padded, B_hat)
    
    C_hat = A_hat * B_hat
    
    c_padded = np.zeros(size, dtype=complex)
    
    IDFT(size, C_hat, c_padded)
    
    c = np.real(c_padded)
    
    return c[:n + m + 1]

#### (d) (4 pts) Use the procedure **multiplyPolys** to compute the product of the two polynomials below:

$p(x) = x^7 − 70.1x^6 + 2.4x^5 − 3.7x^4 + 7.4x^3 − 10.8x^2 + 10.8x − 6.8$

$q(x) = x^8 − 170x^6 + 0.614x^5 + 7392x^4 + 104.2x^3 − 39712x^2 + 51200$

In [8]:
p = [1, -70.1, 2.4, -3.7, 7.4, -10.8, 10.8, -6.8]
q = [1, 0, -170, 0.614, 7392, 104.2, -39712, 0, 51200]

n = 7
m = 8

product = multiplyPolys(n, m, p, q)

print("Product coefficients:", product)

Product coefficients: [ 9.99999999e-01 -7.01000000e+01 -1.67600000e+02  1.19139140e+04
  6.94835860e+03 -5.17455326e+05 -3.05250918e+04  2.75854462e+06
  8.36382880e+03 -3.52008549e+06 -9.22847352e+04  1.90309360e+05
 -5.07181600e+04 -2.82918400e+05  5.52960000e+05 -3.48160000e+05]


##### Outputs:

Product coefficients:
[ 9.99999999e-01, -7.01000000e+01, -1.67600000e+02,  1.19139140e+04,
  6.94835860e+03, -5.17455326e+05, -3.05250918e+04,  2.75854462e+06,
  8.36382880e+03, -3.52008549e+06, -9.22847352e+04,  1.90309360e+05,
 -5.07181600e+04, -2.82918400e+05,  5.52960000e+05, -3.48160000e+05]

### 3. (a) (18 pts) Write code to find the real roots of cubic and quartic polynomials. You should use their closed forms given in the notes.

In [13]:
def solve_cubic(poly: list):
    # normalize
    p = poly[1] / poly[0]
    q = poly[2] / poly[0]
    r = poly[3] / poly[0]

    a = (3*q - p**2) / 3
    b = (2*p**3 - 9*p*q + 27*r) / 27
    delta = (b**2 / 4) + (a**3 / 27)

    if delta > 0:
        A = ((-b / 2) + cmath.sqrt(delta))**(1 / 3)
        B = ((-b / 2) - cmath.sqrt(delta))**(1 / 3)
        y1 = A + B
        y2 = -(y1)/2 + ((cmath.sqrt(3) * 1j) / 2) * (A - B)
        # y3 = -(y1)/2 - ((cmath.sqrt(3) * 1j) / 2) * (A - B)

        return [(y1 - p / 3).real, y2 - p / 3, y2 - p / 3]
    
    elif delta == 0:
        if b > 0:
            y = [-2 * cmath.sqrt(-a/3), cmath.sqrt(-a/3), cmath.sqrt(-a/3)]
        elif b < 0:
            y = [2 * cmath.sqrt(-a/3), -cmath.sqrt(-a/3), -cmath.sqrt(-a/3)]
        elif b == 0:
            y = [0, 0, 0]

        return [(y[0] - p / 3).real, (y[1] - p / 3).real, (y[2] - p / 3).real]
    else:
        theta = cmath.acos((b**2 / 4) / cmath.sqrt(-(a / 3)**3))
        if b > 0:
            theta = -theta

        y1 = 2 * (-a / 3)**0.5 * cmath.cos(theta / 3)
        y2 = 2 * (-a / 3)**0.5 * cmath.cos((theta + 2 * cmath.pi) / 3)
        y3 = 2 * (-a / 3)**0.5 * cmath.cos((theta + 4 * cmath.pi) / 3)
        return [(y1 - p / 3).real, (y2 - p / 3).real, (y2 - p / 3).real]

def solve_quartic(poly: list):
    # normalize
    p = poly[1] / poly[0]
    q = poly[2] / poly[0]
    r = poly[3] / poly[0]
    s = poly[4] / poly[0]

    cubic_roots = solve_cubic([1, -q, p*r - 4*s, 4*q*s - (r**2) - (p**2)*s])
    z1 = cubic_roots[0]

    R = cmath.sqrt(0.25 * p**2 - q + z1)
    if R == 0:
        D = cmath.sqrt(0.75 * p**2 - 2*q + 2 * cmath.sqrt(z1**2 - 4*s))
        E = cmath.sqrt(0.75 * p**2 - 2*q - 2 * cmath.sqrt(z1**2 - 4*s))
    else:
        D = cmath.sqrt(0.75 * p**2 - R**2 - 2*q + 0.25*(4*q*r - 8*r - p**3) / R)
        E = cmath.sqrt(0.75 * p**2 - R**2 - 2*q - 0.25*(4*q*r - 8*r - p**3) / R)

    x1 = -p / 4 + (R + D) / 2
    x2 = -p / 4 + (R - D) / 2
    x3 = -p / 4 - (R - E) / 2
    x4 = -p / 4 - (R + E) / 2

    return [x1, x2, x3, x4]

def solve_quartic_with_np_roots(poly: list):
    # normalize
    p = poly[1] / poly[0]
    q = poly[2] / poly[0]
    r = poly[3] / poly[0]
    s = poly[4] / poly[0]

    cubic_roots = np.roots([1, -q, p*r - 4*s, 4*q*s - (r**2) - (p**2)*s])
    z1 = cubic_roots[0]

    R = cmath.sqrt(0.25 * p**2 - q + z1)
    if R == 0:
        D = cmath.sqrt(0.75 * p**2 - 2*q + 2 * cmath.sqrt(z1**2 - 4*s))
        E = cmath.sqrt(0.75 * p**2 - 2*q - 2 * cmath.sqrt(z1**2 - 4*s))
    else:
        D = cmath.sqrt(0.75 * p**2 - R**2 - 2*q + 0.25*(4*q*r - 8*r - p**3) / R)
        E = cmath.sqrt(0.75 * p**2 - R**2 - 2*q - 0.25*(4*q*r - 8*r - p**3) / R)

    x1 = -p / 4 + (R + D) / 2
    x2 = -p / 4 + (R - D) / 2
    x3 = -p / 4 - (R - E) / 2
    x4 = -p / 4 - (R + E) / 2

    return [x1, x2, x3, x4]

### (b) (4 pts) Run your code on the following two polynomials:
$$p(x) = 110x^3 − 23x^2 + 87x + 4$$
$$q(x) = 43x^4 + 1.34x^3 − 7x^2 − 3400$$

In [14]:
p = [110, -23, 87, 4]
q = [43, 1.34, -7, 0, -3400]

p_roots = solve_cubic(p)
q_roots = solve_quartic(q)
q_roots_with_np = solve_quartic_with_np_roots(q)

print(f"p roots: {p_roots}")
print(f"q roots: {q_roots}")
print(f"q roots using np roots for cubic: {q_roots_with_np}")

p roots: [0.8088582359998703, (-0.7269710052678355+0.3935534423483643j), (0.12720367835887453+0.09960454117848067j)]
q roots: [(1.9596428490947195+1.9465913304769074j), (1.9596428490947195-1.9465913304769074j), (-1.9752242444435568+1.9465910835410145j), (-1.9752242444435568-1.9465910835410145j)]
q roots using np roots for cubic: [(2.9878844886852822+8.512459626075497e-10j), (-0.0077905921839942095+2.968318647290504j), (-0.007790590465811724-2.9683186472905043j), (-3.003466096733151-8.512457405629448e-10j)]


##### Outputs:

p roots: $[0.8088582359998702, (0.12720367835887447-0.0996045411784805j), (0.12720367835887447-0.0996045411784805j)]$

q roots: $[(1.9596428490947195+1.9465913304769074j), (1.9596428490947195-1.9465913304769074j), (-1.9752242444435568+1.9465910835410145j), (-1.9752242444435568-1.9465910835410145j)]$

q roots using np roots for cubic: $[(2.9878844886852822+8.512459626075497e-10j), (-0.0077905921839942095+2.968318647290504j), (-0.007790590465811724-2.9683186472905043j), (-3.003466096733151-8.512457405629448e-10j)]$

*NOTE: I was having issues with solve_cubic. the solve_quartic_with_np_roots uses np.roots to find the cubic roots, so solve_quartic works (it may not seem to work because I couldn't get solve_cubic to work)*

### 4. (a) (29 pts) Implement Muller’s method. Your code must be able to handle complex arithmetic. It should carry out deflation and root polishing via Newton’s method. Also, use the solver implemented in Problem 3 as a subroutine.

In [11]:
def muller_method(f, x0, x1, x2, tol=1e-6):
    while True:
        f0, f1, f2 = f(x0), f(x1), f(x2)
        h1, h2 = x1 - x0, x2 - x1
        δ1, δ2 = (f1 - f0) / h1, (f2 - f1) / h2
        a = (δ2 - δ1) / (h2 + h1)
        b = a * h2 + δ2
        c = f2
        rad = cmath.sqrt(b * b - 4 * a * c)
        
        denom = b + rad if abs(b + rad) > abs(b - rad) else b - rad
        dx = -2 * c / denom
        x3 = x2 + dx
        
        if abs(dx) < tol:
            return x3
        
        x0, x1, x2 = x1, x2, x3

def deflate_polynomial(coeffs, root):
    n = len(coeffs)
    if abs(root.imag) < 1e-6:
        new_coeffs = [0] * (n - 1)
        new_coeffs[0] = coeffs[0]
        for i in range(1, n - 1):
            new_coeffs[i] = coeffs[i] + new_coeffs[i - 1] * root.real
        return new_coeffs
    else:
        real_part, imag_part = root.real, root.imag
        quadratic = [1, -2 * real_part, real_part ** 2 + imag_part ** 2]
        new_coeffs = coeffs.copy()
        for _ in range(2):  # Deflate twice
            new_coeffs = deflate_polynomial(new_coeffs, root)
        return new_coeffs

def newton_method(f, df, x0, tol=1e-6):
    while True:
        fx = f(x0)
        dfx = df(x0)
        if abs(dfx) < tol:
            raise ValueError("Derivative too small for Newton's method")
        dx = -fx / dfx
        x0 += dx
        if abs(dx) < tol:
            return x0

def find_polynomial_roots(coeffs, tol=1e-6):
    degree = len(coeffs) - 1
    roots = []
    
    while degree > 4:
        x0, x1, x2 = 0.5, 1.0, 1.5
        
        f = lambda x: sum(c * x ** i for i, c in enumerate(reversed(coeffs)))
        df = lambda x: sum(i * c * x ** (i - 1) for i, c in enumerate(reversed(coeffs)) if i > 0)
        
        root = muller_method(f, x0, x1, x2, tol)
        
        root = newton_method(f, df, root, tol)
        roots.append(root)
        
        coeffs = deflate_polynomial(coeffs, root)
        degree -= 1 if abs(root.imag) < tol else 2

    if degree == 3:
        roots.extend(solve_cubic(coeffs))
    elif degree == 4:
        roots.extend(solve_quartic(coeffs))
    
    return roots


Roots: [(1.3247179572447454+0j), (-0.6623589786223729+0.5622795120623011j), (-0.6623589786223729-0.5622795120623011j)]


#### (b) (8 pts) Use Muller’s method to find all the zeros, real and complex, of the following two polynomials:
$$p(x) = x^5 − 3.7x^4 + 7.4x^3 − 10.8x^2 + 10.8x − 6.8$$
$$q(x) = x^9 − 0.843121x^8 − 8.35979x^7 + 10.1887x^6 + 14.6196x^5 − 25.7634x^4 + 9.15636x^3 − 0.360995x^2 − 0.180591x + 0.00787276$$

In [12]:
# Coefficients for p(x) = x^5 − 3.7x^4 + 7.4x^3 − 10.8x^2 + 10.8x − 6.8
p_coefficients = [1, -3.7, 7.4, -10.8, 10.8, -6.8]

# Coefficients for q(x) = x^9 − 0.843121x^8 − 8.35979x^7 + 10.1887x^6 + 14.6196x^5 − 25.7634x^4 + 9.15636x^3 − 0.360995x^2 − 0.180591x + 0.00787276
q_coefficients = [1, -0.843121, -8.35979, 10.1887, 14.6196, -25.7634, 9.15636, -0.360995, -0.180591, 0.00787276]

p_roots = find_polynomial_roots(p_coefficients)
print("Roots of p(x):", p_roots)

q_roots = find_polynomial_roots(q_coefficients)
print("Roots of q(x):", q_roots)

Roots of p(x): [(1.700000000000001+0j), (1.5429261246342203+1.748377938368959j), (1.5429261246342203-1.748377938368959j), (-0.5429261246342207+1.2722281192906468j), (-0.5429261246342207-1.2722281192906468j)]
Roots of q(x): [(1.1866744600468007+0j), (1.5764705380452784+0j), (0.26742187571918064+8.17890986533145e-26j), (0.32100086315608634+0j), (0.043486566652317434+0j), (2.8298530156841935+0j), (-0.3354197997621058+0j), (-2.5231832597708754+1.9355815357762378j), (-2.5231832597708754-1.9355815357762378j)]


##### Outputs:

Roots of p(x): $[(1.700000000000001+0j), (1.5429261246342203+1.748377938368959j), (1.5429261246342203-1.748377938368959j), (-0.5429261246342207+1.2722281192906468j), (-0.5429261246342207-1.2722281192906468j)]$

Roots of q(x): $[(1.1866744600468007+0j), (1.5764705380452784+0j), (0.26742187571918064+8.17890986533145e-26j), (0.32100086315608634+0j), (0.043486566652317434+0j), (2.8298530156841935+0j), (-0.3354197997621058+0j), (-2.5231832597708754+1.9355815357762378j), (-2.5231832597708754-1.9355815357762378j)]$